<a href="https://colab.research.google.com/github/XimenaRamirezA/AI-FIME-2024/blob/main/Analisis%20de%20pat%C3%B3genos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1- Carga de Datos

In [ ]:
# Importación de librerías
import os  # Para manejo de archivos y directorios
import cv2  # OpenCV para procesamiento de imágenes
import numpy as np  # Manipulación de arrays numéricos
import matplotlib.pyplot as plt  # Visualización de resultados
from sklearn.model_selection import train_test_split  # División de datos
from sklearn.preprocessing import LabelEncoder  # Codificación de etiquetas
from keras.utils import to_categorical  # Codificación one-hot
import tensorflow as tf  # Framework para deep learning
from tensorflow.keras import layers, models  # Componentes para construir la red



#2- Carga y Preprocesamiento de Imagenes

In [ ]:
# Configuración de rutas y parámetros
DATA_DIR = '/content/microscopy_dataset'  # Ruta del dataset de imágenes
IMG_SIZE = (164, 164)  # Tamaño al que se redimensionarán las imágenes

# Listas para almacenar imágenes y etiquetas
images = []
labels = []

# Procesamiento de las imágenes
for category in sorted(os.listdir(DATA_DIR)):
    category_path = os.path.join(DATA_DIR, category)
    if os.path.isdir(category_path):
        for img_name in os.listdir(category_path):
            img_path = os.path.join(category_path, img_name)
            img = cv2.imread(img_path)  # Carga la imagen

            if img is not None:
                # Preprocesamiento:
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convertir a escala de grises
                resized = cv2.resize(gray, IMG_SIZE)  # Redimensionar
                images.append(resized)
                labels.append(category)  # Almacena la etiqueta

# Preparación de los datos para el modelo
images = np.array(images).reshape(-1, 164, 164, 1) / 255.0  # Normalización (0-1)
labels = np.array(labels)  # Convertir a array numpy


# 3- Preparación de los datos para el modelo

In [ ]:
# Codificación de etiquetas
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)  # Convertir etiquetas a números
labels_onehot = to_categorical(labels_encoded)  # Codificación one-hot (opcional)

# División del dataset (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(
    images, labels_encoded, test_size=0.2, random_state=42)
num_classes = len(np.unique(labels_encoded))  # Número de clases


# 4- Construcción del modelo CNN y Compilacion del modelo

In [ ]:
# Construcción del modelo CNN
model = models.Sequential([
    # Capa convolucional 1
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(164, 164, 1)),
    layers.MaxPooling2D((2, 2)),  # Reducción dimensional

    # Capa convolucional 2
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Capa convolucional 3
    layers.Conv2D(64, (3, 3), activation='relu'),

    # Capas densas (fully connected)
    layers.Flatten(),  # Aplanar para conexión densa
    layers.Dense(64, activation='relu'),  # Capa oculta
    layers.Dense(num_classes, activation='softmax')  # Capa de salida
])

# Compilación del modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Función de pérdida
              metrics=['accuracy'])  # Métrica a monitorear

# Entrenamiento del modelo
history = model.fit(X_train, y_train,
                   epochs=10,  # Número de iteraciones
                   validation_data=(X_test, y_test))  # Datos de validación



# 6. Visualización y evaluación

In [ ]:
# Visualización del rendimiento
plt.plot(history.history['accuracy'], label='Precisión en entrenamiento')
plt.plot(history.history['val_accuracy'], label='Precisión en validación')
plt.xlabel('Época')
plt.ylabel('Precisión')
plt.legend()
plt.grid()
plt.show()


In [ ]:
# Prueba con una imagen del conjunto de test
index = 0  # Índice de la imagen a probar
sample = np.expand_dims(X_test[index], axis=0)  # Preparar muestra para predicción
prediction = np.argmax(model.predict(sample))  # Obtener predicción
true_label = y_test[index]  # Etiqueta real

# Mostrar resultados
print(f"Predicción del modelo: {le.inverse_transform([prediction])[0]}")
print(f"Etiqueta real: {le.inverse_transform([true_label])[0]}")

# Visualizar la imagen
plt.imshow(X_test[index].reshape(164, 164), cmap='gray')
plt.axis('off')
plt.show()
